In [2]:
import jdc
import pandas as pd
import ftplib as ftp
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [3]:
class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [4]:
class taxa_sample(object):
    def __init__(self,
                 assembly_summary='/work/ncbi_dbs/assembly_summary_genbank.txt'):
        
        header = 'assembly_accession bioproject biosample wgs_master refseq_category\
          taxid species_taxid organism_name infraspecific_name isolate version_status\
          assembly_level release_type genome_rep seq_rel_date asm_name submitter\
          gbrs_paired_asm paired_asm_comp ftp_path excluded_from_refseq relation_to_type_material'.split()
        self.assembly_summary = pd.read_table(assembly_summary,
                                         comment='#', header=None, names=header,
                                         dtype={'taxid':str, 'infraspecific_name':str})
        self.assembly_summary['refseq_category']  = self.assembly_summary['refseq_category'].str.lower()
        self.assembly_summary['assembly_level']   = self.assembly_summary['assembly_level'].str.lower()
        self.assembly_summary['genome_rep']       = self.assembly_summary['genome_rep'].str.lower()
        self.assembly_summary.set_index('assembly_accession', inplace=True)
        
        self.refseq_summary = pd.read_table('/work/ncbi_dbs/assembly_summary_refseq.txt',
                                         comment='#', header=None, names=header,
                                         dtype={'taxid':str, 'infraspecific_name':str})
        self.refseq_summary['refseq_category']  = self.refseq_summary['refseq_category'].str.lower()
        self.refseq_summary['assembly_level']   = self.refseq_summary['assembly_level'].str.lower()
        self.refseq_summary['genome_rep']       = self.refseq_summary['genome_rep'].str.lower()
        self.refseq_summary.set_index('assembly_accession', inplace=True)
        self.genome_taxids = None

In [5]:
%%add_to taxa_sample
def download_genome(self, sub_table, destination_folder='.'):
    ncbi_ftp = ftp.FTP('ftp.ncbi.nlm.nih.gov')
    ncbi_ftp.login()
    succesful_download = []
    for index, row in sub_table.iterrows():
        ncbi_ftp.cwd('/')
        path = row['ftp_path'].replace('ftp://ftp.ncbi.nlm.nih.gov/', '')
        try:
            ncbi_ftp.cwd(path)
        except:
            continue
        assembly_files = ncbi_ftp.nlst()
        for assembly_file in assembly_files:
            if assembly_file.endswith('protein.faa.gz'):
                handle = open('%s/%s.gz' % (destination_folder, index), 'wb')
                ncbi_ftp.retrbinary("RETR %s" % assembly_file, handle.write)
                succesful_download.append(index)
    ncbi_ftp.quit()
    return succesful_download

In [6]:
%%add_to taxa_sample
def prune_taxa(self, accession_numbers, assembly_level='contig', genome_rep='partial'):
    tmp_df = self.assembly_summary.loc[accession_numbers,
                                       'assembly_level genome_rep'.split()].copy()
    tmp_df = tmp_df[(tmp_df.assembly_level != assembly_level) &
                    (tmp_df.genome_rep     != genome_rep)]

    return tmp_df.index.tolist()

In [7]:
%%add_to taxa_sample
def genomes_from_taxon(self, target_taxon):
    if not self.genome_taxids:
        self.genome_taxids = ','.join(set(self.assembly_summary.taxid.tolist()))
    query  = ncbi.db.execute("SELECT taxid FROM species WHERE taxid IN (%s) AND \
                             ',' || track || ',' like '%%,%s,%%';" %
                             (self.genome_taxids, target_taxon))
    found_taxids = [str(x[0]) for x in query.fetchall()]
    return self.assembly_summary.index[
        self.assembly_summary.taxid.isin(found_taxids)
    ].tolist()

In [130]:
import os
import shutil
import subprocess
import re
import numpy as np
import time
from Bio import Entrez, SeqIO
Entrez.email = "lthiberiol@gmail.com"

genome_sample = taxa_sample()
%cd /work/kelsey/

/work/kelsey


/Users/thiberio/anaconda2/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [142]:
genomes = pd.read_excel('/work/kelsey/Kelsey_CYPL_key_accessionNumbers_modified_4-2-19.xlsx',
                        header=None,
                        names='Organism abbreviation accession'.split())

In [143]:
sample_accession   = genomes.query('accession==accession and accession.str.startswith("SAM")').accession.tolist()
project_accession  = genomes.query('accession==accession and accession.str.startswith("PRJ")').accession.tolist()
assembly_accession = genomes.query('accession==accession and accession.str.startswith("GC")').accession.tolist()

In [144]:
len(sample_accession)+len(project_accession)+len(assembly_accession)

142

In [10]:
missing_prj = [x for x in project_accession
                  if x not in genome_sample.assembly_summary.bioproject.tolist()]
for bioprj in missing_prj:
    print(bioprj)
    handle = Entrez.esearch(db="nucleotide", term=bioprj)
    record = Entrez.read(handle)
    
    gb     = Entrez.efetch(db='nucleotide',
                           id=record['IdList'][0],
                           rettype="gb",
                           retmode="text").read()
    out    = open('hgt/missing_bioprj/%s.gb' % bioprj, 'w')
    out.write(gb)
    out.close()
    time.sleep(3)

PRJNA12230
PRJNA325991
PRJNA12239
PRJNA238126
PRJNA356572
PRJNA19853
PRJNA21061
PRJNA15743
PRJNA50277
PRJNA41869
PRJNA20387
PRJNA66177
PRJNA314878
PRJNA13599
PRJNA267422
PRJNA50237
PRJNA291892
PRJDB3738
PRJNA12234
PRJNA218133
PRJNA12229
PRJNA45999
PRJNA18283
PRJNA28131
PRJNA232187
PRJNA287360
PRJNA16670
PRJNA344076
PRJNA20561
PRJNA81371
PRJNA17049


In [145]:
genbank_genomes = genome_sample.assembly_summary.query(
    'biosample.isin(@sample_accession) or \
     bioproject.isin(@project_accession) or \
     index.isin(@assembly_accession)')

refseq_genomes = genome_sample.refseq_summary.query(
    'biosample.isin(@sample_accession) or \
     bioproject.isin(@project_accession) or \
     index.isin(@assembly_accession)').copy()
refseq_genomes.drop(genbank_genomes.gbrs_paired_asm,
                    errors='ignore', inplace=True)

In [146]:
ncbi_genomes = genbank_genomes.append(refseq_genomes)
ncbi_genomes['assembly_accession'] = ncbi_genomes.index

In [147]:
tmp1 = ncbi_genomes.merge(genomes, left_on='biosample' , right_on='accession')
tmp2 = ncbi_genomes.merge(genomes, left_on='bioproject', right_on='accession')
tmp3 = ncbi_genomes.merge(genomes, left_index=True     , right_on='accession')

In [148]:
full_genome_df = tmp1.append(tmp2)
full_genome_df = full_genome_df.append(tmp3)
full_genome_df.set_index('assembly_accession', inplace=True)
full_genome_df.head()

,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,...,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,Organism,abbreviation,accession
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCA_000006985.1,PRJNA302,SAMN02604006,NaN,reference genome,194439,1097,Chlorobaculum tepidum TLS,strain=TLS,NaN,latest,...,ASM698v1,TIGR,GCF_000006985.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,Chl_194439.7 (Chlorobium_tepidum),Chl0,SAMN02604006
GCA_000007925.1,PRJNA419,SAMN02603142,NaN,reference genome,167539,1219,Prochlorococcus marinus subsp. marinus str. CC...,strain=CCMP1375; SS120,NaN,latest,...,ASM792v1,CNRS,GCF_000007925.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,Prochlorococcus_marinus_str_CCMP1375,P_CCMP1375,SAMN02603142
GCA_000011385.1,PRJNA9606,SAMD00061120,NaN,reference genome,251221,33072,Gloeobacter violaceus PCC 7421,strain=PCC 7421,NaN,latest,...,ASM1138v1,Kazusa,GCF_000011385.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,Gloeobacter_violaceus_PCC_7421,G_violaceu,SAMD00061120
GCA_000011485.1,PRJNA220,SAMEA3138210,NaN,representative genome,74547,1219,Prochlorococcus marinus str. MIT 9313,strain=MIT9313,NaN,latest,...,ASM1148v1,DOE Joint Genome Institute,GCF_000011485.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,Prochlorococcus_marinus_MIT_9313,P_marinus,SAMEA3138210
GCA_000012345.1,PRJNA13989,SAMN02603690,NaN,na,335992,198252,Candidatus Pelagibacter ubique HTCC1062,strain=HTCC1062,NaN,latest,...,ASM1234v1,Oregon State University,GCF_000012345.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,Candidatus_Pelagibacter,Cand_Pelag,SAMN02603690


In [149]:
missing_bioprj = [filename.replace('.gb', '') for filename in os.listdir('hgt/missing_bioprj/')]
missing_bioprj = genomes[genomes.accession.isin(missing_bioprj)].copy()

In [150]:
full_genome_df = full_genome_df.append(missing_bioprj, sort=False)

In [151]:
full_genome_df.to_csv('genomes.tab', sep='\t')

In [23]:
genbank_genomes.shape, refseq_genomes.shape, ncbi_genomes.shape

((107, 21), (3, 21), (110, 21))

In [14]:
downloaded_genomes = []
with cd('hgt/genomes/'):
    downloaded_genomes.extend(genome_sample.download_genome(genbank_genomes, destination_folder='../sources/'))
    downloaded_genomes.extend(genome_sample.download_genome(refseq_genomes, destination_folder='../sources/'))
    for genome in downloaded_genomes:
        subprocess.call(['gunzip', '../sources/%s.gz' % genome])


        fasta = open('../sources/%s' % genome).readlines()
        out   = open('%s.faa' % genome, 'w')
        for line in fasta:
            if line.startswith('>'):
                sequence_acc = re.search('^>(\S+)', line, re.M).group(1)
                row = None
                try:
                    row = genome_sample.assembly_summary.loc[genome]
                except KeyError:
                    row = genome_sample.refseq_summary.loc[genome]

                organism = row['organism_name']

                strain = ''
                if pd.notnull(row['infraspecific_name']):
                    strain  = row['infraspecific_name'].replace('strain=', '')
                if not organism.endswith(strain):
                    organism = '%s %s' % (organism, strain)
                new_header   = '>%s|%s [%s]\n' % (sequence_acc, genome, organism)

                out.write(new_header)
            else:
                out.write(line)
        out.close()

In [15]:
with cd('hgt/missing_bioprj/'):
    for filename in os.listdir('.'):
        gb    = SeqIO.read(filename, 'gb')
        accession = filename.replace('.gb', '')
        fasta = open('../genomes/%s.faa' % accession, 'w')
        for feature in gb.features:
            if feature.type != 'CDS':
                continue
            try:
                seq_id = feature.qualifiers['locus_tag'][0]
            except KeyError:
                seq_id = feature.qualifiers['protein_id'][0]
            
            fasta.write('>%s|%s [%s]\n%s\n' % 
                       (seq_id,
                        accession,
                        gb.annotations['organism'],
                        feature.qualifiers['translation'][0]))
        fasta.close()

In [23]:
set(os.listdir('hgt/genomes-old//')).difference(os.listdir('hgt/genomes/'))

set()